In [1]:
%install-location $cwd/swift-install
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Batcher ModelSupport Datasets

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))
		Batcher
		ModelSupport
		Datasets
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp807a4l6c/swift-install
/home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift-build: /home/sudhirsuman/anaconda3/lib/libcurl.so.4: no version information available (required by /home/sudhirsuman/Desktop/swiftPersonal/usr/lib/swift/linux/libFoundationNetworking.so)
/home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift: /home/sudhirsuman/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift)
/home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift: /home/sudhirsuman/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift)
/home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift: /home/sudhirsuman/anaconda3/lib/libuuid.so.1: no version information available (re

In [2]:
import Foundation
import ModelSupport
import TensorFlow
import Datasets

# Dataset

In [3]:
extension Sequence where Element : Collection {
    subscript(column column : Element.Index) -> [ Element.Iterator.Element ] {
        return map { $0[ column ] }
    }
}
extension Sequence where Iterator.Element: Hashable {
    func unique() -> [Iterator.Element] {
        var seen: Set<Iterator.Element> = []
        return filter { seen.insert($0).inserted }
    }
}

In [178]:
public struct MovieLens {

    public let user_pool: [Float]
    public let item_pool: [Float]
    public let num_users: Int
    public let num_items: Int
    public let user_item_rating: [[Int]]
    public let rating: [Float]
    public let user2id: [Float:Int]
    public let id2user: [Int:Float]
    public let item2id: [Float:Int]
    public let id2item: [Int:Float]
    public let neg_sampling: Tensor<Float>

    static func downloadMovieLensDatasetIfNotPresent() -> String{
        let localURL = URL(fileURLWithPath: FileManager.default.currentDirectoryPath)
        let dataFolder = DatasetUtilities.downloadResource(
            filename: "ml-1m",
            fileExtension: "zip",
            remoteRoot: URL(string: "http://files.grouplens.org/datasets/movielens/")!,
            localStorageDirectory: localURL.appendingPathComponent("data/", isDirectory: true))

        return try! String(contentsOf: dataFolder.appendingPathComponent("ratings.dat"), encoding: .utf8)
    }

    public init() {
        let data  = MovieLens.downloadMovieLensDatasetIfNotPresent()
        let df_data: [[Float]] = data.split(separator: "\n").map{ String($0).split(separator: ":").compactMap{ Float(String($0)) } }
        
        var df = df_data[0..<30000]
        let user_pool = df[column: 0].unique()
        let item_pool = df[column: 1].unique()
        let rating = df[column: 2]

        let user_index = 0...user_pool.count-1
        let user2id:[Float:Int] = Dictionary(uniqueKeysWithValues: zip(user_pool,user_index))
        let id2user:[Int:Float] = Dictionary(uniqueKeysWithValues: zip(user_index,user_pool))

        let item_index = 0...item_pool.count-1
        let item2id:[Float:Int] = Dictionary(uniqueKeysWithValues: zip(item_pool,item_index))
        let id2item:[Int:Float] = Dictionary(uniqueKeysWithValues: zip(item_index,item_pool))

        let preprocess_rating = rating.map({ $0 > 0  ? 1 : $0 })
        var neg_sampling = Tensor<Float>(zeros: [user_pool.count,item_pool.count])

        for element in df{
            let u_index = user2id[element[0]]!
            let i_index = item2id[element[1]]!
            neg_sampling[u_index][i_index] = Tensor(1.0)
        }
        
        var dataset:[[Int]] = []
        for user_id in user_index{
            for item_id in item_index{
                let rating  = Int(neg_sampling[user_id][item_id].scalarized())
                dataset.append([user_id,item_id, rating])
            }
        }
    
        self.num_users = user_pool.count
        self.num_items = item_pool.count
        self.user_pool = user_pool
        self.item_pool = item_pool
        self.rating = rating
        self.user2id = user2id
        self.id2user = id2user
        self.item2id = item2id
        self.id2item = id2item
        self.user_item_rating = dataset
        self.neg_sampling = neg_sampling
    }
}

# Model

In [169]:
public struct NeuMF: Module {

    public typealias Scalar = Float

    @noDerivative public let num_users: Int
    @noDerivative public let num_items: Int
    @noDerivative public let mf_dim: Int
    @noDerivative public let mf_reg: Scalar
    @noDerivative public var mlp_layer_sizes : [Int] = [64,32,16,8]
    @noDerivative public var mlp_layer_regs: [Scalar] = [0,0,0,0]

    public var mf_user_embed: Embedding<Scalar>
    public var mf_item_embed: Embedding<Scalar>
    public var mlp_user_embed: Embedding<Scalar>
    public var mlp_item_embed: Embedding<Scalar>
    public var dense1: Dense<Scalar>
    public var dense2: Dense<Scalar>
    public var dense3: Dense<Scalar>
    public var final_dense: Dense<Scalar>


    public init(
        num_users: Int,
        num_items: Int,
        mf_dim: Int,
        mf_reg: Float,
        mlp_layer_sizes: [Int],
        mlp_layer_regs: [Float]
    ) {
        self.num_users = num_users
        self.num_items = num_items
        self.mf_dim = mf_dim
        self.mf_reg = mf_reg
        self.mlp_layer_sizes = mlp_layer_sizes
        self.mlp_layer_regs = mlp_layer_regs

        // precondition(self.mlp_layer_sizes[0]%2 == 0, "u dummy, mlp_layer_sizes[0] % 2 != 0")
        // precondition(self.mlp_layer_sizes.count == self.mlp_layer_regs.count, "u dummy, layer_sizes != layer_regs!")

        //TODO: regularization
        self.mf_user_embed = Embedding<Scalar>(vocabularySize: self.num_users, embeddingSize: self.mf_dim)
        self.mf_item_embed = Embedding<Scalar>(vocabularySize: self.num_items, embeddingSize: self.mf_dim)
        self.mlp_user_embed = Embedding<Scalar>(vocabularySize: self.num_users, embeddingSize: self.mlp_layer_sizes[0]/2)
        self.mlp_item_embed = Embedding<Scalar>(vocabularySize: self.num_items, embeddingSize: self.mlp_layer_sizes[0]/2)

        //TODO: Extend it for n layers by using for loop
        //Currently only for 3 layers
        dense1 = Dense(inputSize: self.mlp_layer_sizes[0], outputSize: self.mlp_layer_sizes[1], activation: relu)
        dense2 = Dense(inputSize: self.mlp_layer_sizes[1], outputSize: self.mlp_layer_sizes[2], activation: relu)
        dense3 = Dense(inputSize: self.mlp_layer_sizes[2], outputSize: self.mlp_layer_sizes[3], activation: relu)
        final_dense = Dense(inputSize: (self.mlp_layer_sizes[3] + self.mf_dim), outputSize: 1)
    }
        @differentiable
        public func callAsFunction(_ input: Tensor<Int32>) -> Tensor<Scalar>{
            let user_indices  = input[0]
            let item_indices = input[1]

            let user_embed_mlp = self.mlp_user_embed(user_indices)
            let item_embed_mlp = self.mlp_item_embed(item_indices)
            let user_embed_mf = self.mf_user_embed(user_indices)
            let item_embed_mf = self.mf_item_embed(item_indices)

            // let mf_vector = matmul(user_embed_mf,item_embed_mf)
            let mf_vector = user_embed_mf*item_embed_mf
            var mlp_vector = user_embed_mlp.concatenated(with:item_embed_mlp,alongAxis:-1)
            //
            // print(mlp_vector.shape)
            mlp_vector = mlp_vector.sequenced(through: dense1, dense2, dense3)
            let vector = mlp_vector.concatenated(with:mf_vector,alongAxis:-1)

            return final_dense(vector)
            // return mf_vector
        }
    // }
}


# Training

In [ ]:
let dataset = MovieLens()

In [170]:
let num_users = dataset.num_users
let num_items = dataset.num_items

In [171]:
var size:[Int] = [64, 32, 16, 8]
var regs:[Float] = [0, 0, 0, 0]
var model = NeuMF(num_users: num_users, num_items: num_items, mf_dim: 10, mf_reg: 0.0, mlp_layer_sizes: size, mlp_layer_regs: regs)

In [172]:
let optimizer = Adam(for: model, learningRate: 0.001)
let train_matrix = dataset.neg_sampling
for epoch in 1...3{
    var avg_loss: Float = 0.0
    for (i,data) in user_item_rating.enumerated(){
            let user_id = data[0]
            let item_id = data[1]
            let rating = Tensor(Float(data[2]))
        
            let input = Tensor<Int32>(shape: [2, 1], scalars: [Int32(user_id), Int32(item_id)])
            let (loss, grad) = valueWithGradient(at: model) { model -> Tensor<Float> in
            let logits = model(input)
            return sigmoidCrossEntropy(logits: logits, labels:rating)
//             return meanSquaredError(predicted: logits, expected: rating)
                                                            }
            optimizer.update(&model, along: grad)
            avg_loss = avg_loss + loss.scalarized()
            
    }
    print("Current loss: \(avg_loss/(Float(num_users*num_items)))")
}

Current loss: 0.39105844
Current loss: 0.40510172
Current loss: 0.3449839


# Testing

In [173]:
var input: Tensor<Int32> = [[4],[1]]
model(input)

[[-0.28371918]]


In [174]:
var output:[Float] = []
for i in 0...num_items-1{
    var input =  Tensor<Int32>(shape: [2, 1],scalars: [3,Int32(i)])
    output.append(model(input).scalarized())
}

In [175]:
let x = exp(Tensor(output))/(exp(Tensor(output)).sum())

In [176]:
x.argmax()

386


In [177]:
x.argmin()

371
